### List of all products 

In [1]:
#!/usr/bin/python
# -*- coding: utf-8 -*-

import json
import pandas as pd
import pprint
pp = pprint.PrettyPrinter(indent=4)

In [2]:
import requests

BASE_URL='https://www.openfood.ch/api/v3'
KEY='97bb347d26f3a8b39b4558827c3fcf56'
ENDPOINT='/products'


'''
url = BASE_URL + ENDPOINT

headers = {
  'Authorization': 'Token token=' + KEY,
  'Accept': 'application/json',
  'Content-Type': 'application/vnd.api+json',
  'Accept-Encoding': 'gzip,deflate'
}
productlist = []
listlength = 0
i = 0
for i in range(0, 2):
    query = {
        'page[number]': i,
        'page[size]': 200  # maximum
    }
    r = requests.get(url, params=query, headers=headers)
    if r.status_code == 200:
        print('Page ' + str(i) + ' loaded successfully.')

        for product in r.json()['data']:
            try:
                productlist.append(product['id'])
            except:
                pass

    if listlength == len(productlist):
        break
    #print('Product count: ' + str(len(productlist)))
    listlength = len(productlist)
    i += 1
    
'''
url = BASE_URL + ENDPOINT

query = {
  'page[number]': 1,
  'page[size]': 200
}

headers = {
  'Authorization': 'Token token=' + KEY,
  'Accept': 'application/json',
  'Content-Type': 'application/vnd.api+json',
  'Accept-Encoding': 'gzip,deflate'
}

r = requests.get(url, params=query, headers=headers)
print("Status: " + str(r.status_code))
if r.status_code == 200:
    print('Page loaded successfully.')
    print('Generated in ' + str(r.json().get('meta', {}).get('generated_in', -1)) + ' milliseconds.')
    print('Next page\'s URL is: ' + r.json()['links'].get('next', 'UNKNOWN'))
    print('Barcodes on this page:')
    for product in r.json()['data']:
        productlist = []
        for x in r.json()['data']:
            productlist.append(x['id'])
print(productlist)

Status: 200
Page loaded successfully.
Generated in 164 milliseconds.
Next page's URL is: https://www.openfood.ch/api/v3/products?page%5Bnumber%5D=2&page%5Bsize%5D=200
Barcodes on this page:
[1000, 10000, 10001, 10002, 10003, 10004, 10005, 10006, 10007, 10008, 10009, 1001, 10010, 10011, 10012, 10013, 10014, 10015, 10016, 10017, 10018, 10019, 1002, 10020, 10021, 10022, 10023, 10024, 10025, 10026, 10027, 10028, 10029, 1003, 10030, 10031, 10032, 10033, 10034, 10035, 10036, 10037, 10038, 10039, 1004, 10040, 10041, 10042, 10043, 10044, 10045, 10046, 10047, 10048, 10049, 1005, 10050, 10051, 10052, 10053, 10054, 10055, 10056, 10057, 10058, 10059, 10060, 10061, 10062, 10063, 10064, 10065, 10066, 10067, 10068, 10069, 10070, 10071, 10072, 10073, 10074, 10075, 10076, 10077, 10078, 10079, 10080, 10081, 10082, 10083, 10084, 10085, 10086, 10087, 10088, 10089, 10090, 10091, 10092, 10093, 10094, 10095, 10096, 10097, 10098, 10099, 1010, 10100, 10101, 10102, 10103, 10104, 10105, 10106, 10107, 10108, 1010

### Getting list of useful characteristics from id

In [3]:
%%time
BASE_URL='https://www.openfood.ch/api/v3'
KEY='97bb347d26f3a8b39b4558827c3fcf56'
dInfo = {}
dIngredients = {}
dNutrients = {}

for PRODUCT_ID in productlist:
    ENDPOINT='/products/{}'.format(PRODUCT_ID)

    url = BASE_URL + ENDPOINT

    headers = {
      'Authorization': 'Token token=' + KEY,
      'Accept': 'application/json',
      'Content-Type': 'application/vnd.api+json',
      'Accept-Encoding': 'gzip,deflate'
    }

    r = requests.get(url, headers=headers)
    #print ('Status: ' + str(r.status_code))
    item = {}
    item_ing = {}
    item_nut = {}
    data = r.json()['data']
    if r.status_code == 200:
        try:
            item['name'] = item_ing['name'] = item_nut['name'] = data['display_name_translations'].popitem()
            item['id'] = item_ing['id'] = item_nut['id'] = data['id']
            #item['barcode'] = item_ing['barcode'] = item_nut['barcode'] = data['barcode']
            
            item['quantity'] = data['quantity']
            item['unit'] = data['unit']
            item['portion'] = data['portion_quantity']
            item['portion unit'] = data['portion_unit']
            item['alcohol'] = data['alcohol_by_volume']
            
            item_ing['ingredients'] = data['ingredients_translations'].popitem()
            
            nuts = {}
            for nut in data['nutrients'].keys():
                nuts[nut] = {'per_day':data['nutrients'][nut]['per_day'],
                             'per_hundred':data['nutrients'][nut]['per_hundred'],
                             'per_portion':data['nutrients'][nut]['per_portion']}
            item_nut['nutrients'] = nuts
            
            dInfo[data['barcode']] = item    
            dIngredients[data['barcode']] = item_ing
            dNutrients[data['barcode']] = item_nut
        except:
            pass

CPU times: user 9.64 s, sys: 324 ms, total: 9.96 s
Wall time: 47.1 s


### Translate ingredients

In [4]:
print(dIngredients.popitem())

('7613312081242', {'name': ('it', 'EL MUNDO GLOBAL CUISINE RATATOUILLE GRILLED STYLE PERU'), 'ingredients': ('it', "pomodori 50%, melanzane 14%, zucchine 10%, peperoni «Piquillo» 7%, carciofi 6%, cipolle, sedano, olio di girasole, zucchero, sale da cucina, fecola di patate, capperi, regolatore d'acidità: acido citrico, antiossidante: acido ascorbico."), 'id': 10035})


In [5]:
%%time
from watson_developer_cloud import LanguageTranslatorV2 as LanguageTranslator

language_translator = LanguageTranslator(username='a69f77dc-9248-4069-85b5-1ac1ef6bfd7d',password='UQznP8xcKtuq')

count = 0
print("%s/%s items done..."%(count, len(dIngredients)))
remove = []
for barcode, tmp in dIngredients.items():
    try:        
        ingredients_ = ' '.join(tmp['ingredients'][1].splitlines())
        language_json = language_translator.identify(ingredients_)
        language = pd.DataFrame(language_json['languages']).sort_values(by=['confidence'], ascending=False)
        source_lang = language.iloc[0, 1]
        if source_lang != "en":
            translation = language_translator.translate(text=ingredients_, source=source_lang, target='en')
            dIngredients[barcode]['ingredients'] = "en_translated, " + json.dumps(translation, indent=2, ensure_ascii=False)
        else:
            dIngredients[barcode]['ingredients'] = "en_kept, " + ingredients_ 
                        
    except:
        print(source_lang, ingredients_)
        remove.append(barcode)
        
    count += 1
    if count % 50 == 0:
        print("%s/%s items done!"%(count, len(dIngredients)))

# remove the ones (from everywhere) which couldn't be translated
for barcode in remove:
    del dInfo[barcode]
    del dIngredients[barcode]
    del dNutrients[barcode]

0/184 items done...
da Guavensaft 100%
50/184 items done!
100/184 items done!
vi Wheat flour (69%), sugar, refined, vegetable fat, dextrose, reduced cocoa powder, salt, raising agents (sodium bicarbonate and ammonium bocarbonate) cinnamon powder and antioxidant (sodium metabisulphite).
150/184 items done!
id Ananas 62% (Indonesien, Asien), Ananassaft.
nl Mais, Wasser, Meersalz, Zitronensaft
CPU times: user 19.9 s, sys: 596 ms, total: 20.5 s
Wall time: 4min 4s


In [6]:
print(dIngredients.popitem())

('7613312035771', {'name': ('it', 'Sun Queen Weisse Spargeln'), 'ingredients': 'en_translated, "White asparagus 61% (Peru), water, white wine vinegar, sugar, salt, antioxidants: Ascorbinsaüre"', 'id': 10027})


In [7]:
# dump dictionaries to json files (load it in with `json.load(fName)`)

with open("dInfo.json", "w") as f_:
    json.dump(dInfo, f_)
with open("dIngredients.json", "w") as f_:
    json.dump(dIngredients, f_)
with open("dNutrients.json", "w") as f_:
    json.dump(dNutrients, f_)

### Calculate climate impact!

In [ ]:
import json

with open("dInfo.json", "r") as f_:
    dInfo = json.load(f_)
with open("dIngredients.json", "r") as f_:
    dIngredients = json.load(f_)
with open("dNutrients.json", "r") as f_:
    dNutrients = json.load(f_)

In [8]:
from food_db import build_database
ingredients_to_impact = build_database()

In [9]:
import re
ingredient_dict = {}

for item in list(dIngredients.items()):
    lItems = item[1]['ingredients'].split(',')[1:]
    quantity_dict = {}
    estimate = 50
    for l in lItems:
        percentage = re.findall(r'\d+.\d+%', l)
        if(len(percentage) == 0):
            quantity_dict[l] = estimate
        else:
            quantity_dict[l] = percentage[0].rstrip('%')
        estimate = estimate/2
    ingredient_dict[item[0]] = quantity_dict

In [11]:
final_dict = {}
for bar_code in list(ingredient_dict):
    temp_dict = {}
    temp_dict['ingredients'] = ingredient_dict[bar_code]
    temp_dict['quantity'] = dInfo[bar_code]['quantity']
    temp_dict['unit'] = dInfo[bar_code]['unit']
    final_dict[bar_code] = temp_dict
    

In [12]:
import difflib
footprint_dict = {}

for item in list(final_dict.items()):
    footprint = 0
    ingredients = item[1]['ingredients']
    quantity = float(final_dict[item[0]]['quantity'])
    for ing_key, ing_quantity in ingredients.items():
        print(ing_key)
        scores = []
        keys = []
        for key, value in ingredients_to_impact.items():
            keys.append(key)
            scores.append(difflib.SequenceMatcher(None, ing_key,key).ratio())
        matched_score = ingredients_to_impact[keys[scores.index(max(scores))]]
        print(quantity,ing_quantity,matched_score)
        footprint += float(quantity)*float(ing_quantity)*0.01*float(matched_score)
    footprint_dict[item[0]] = footprint

 "Quince banana * 45%
0.0 50 0.27
 * 20% * from organic agriculture"
0.0 12.5 1.77
 * 35%
0.0 25.0 0.44
 "Water
230.0 50 0.44
 jalapeño peppers (4%)
230.0 6.25 0.26
 tomatoes (11%)
230.0 12.5 0.23
 corn starch modified
230.0 0.09765625 0.22
 sugar
230.0 0.390625 3.01
 Regulator of acidité (citric acid)"
230.0 0.048828125 3.89
 dried onions
230.0 3.125 0.29
 vinaigre of alcohol
230.0 0.78125 0.34
 tomato paste (12%)
230.0 25.0 0.23
 chili green (0.8%)
230.0 0.8 3.73
 salt from the kitchen
230.0 1.5625 0.26
 "Water
270.0 50 0.44
 flour konjak 3%
270.0 25.0 0.65
 age resistance : E526."
270.0 12.5 2.41
 beans
530.0 25.0 0.76
 Lupin
530.0 50 0.14
 water
530.0 12.5 0.44
 salt and acidity regulator: E330
530.0 6.25 4.51
 "Chilis jalapenos (53.5%)
215.0 53.5 2.88
 vinegar
215.0 12.5 0.34
 water
215.0 25.0 0.44
 salt from the kitchen."
215.0 6.25 0.26
 "Judah"
0.0 50 0.44
 stabilising : And 509
0.0 3.125 1.89
 water
0.0 25.0 0.44
 sugar
0.0 12.5 3.01
 acid
0.0 1.5625 5.25
 "Pears 58% (Southern

In [ ]:
with open("barcode_to_impact.json", "w") as f_:
    json.dump(dInfo, f_)